In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)

2022-11-29 11:07:09.582897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 11:07:09.803571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 11:07:09.803607: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 11:07:11.041211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [5]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.transforms import UMAP
from sklearn.manifold import TSNE
from experiments.Topological_ae.Experiment_utils import *
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

## Reading model

In [7]:
import pickle
import matplotlib.pyplot as plt
from tabulate import tabulate
# Variables to reuse
save_ids = range(5)
model_name = 'ConvolutionalAutoencoder_custom_dim3'
model_dim = 10
model_epc = 500
model_lams = [0, 0.1, 0.01, 1, 10]

data = {
    'Modelo': ['Conv_custom_dim3']*len(model_lams),
    'Lambda': model_lams,
    'RF-ACC': [],
    'RF-F1': [],
    'SVC-ACC': [],
    'SVC-F1': [],
    'KNN-ACC': [],
    'KNN-F1': [],
    }
for model_lam in model_lams:
    rf_acc = []
    rf_f1 = []
    svc_acc = []
    svc_f1 = []
    knn_acc = []
    knn_f1 = []
    recon_error = []
    for save_id in save_ids:
        filename = 'data/{}_{}_{}_{}_{}.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename, 'rb')
        model = pickle.load(filehandler)
        filehandler.close()
        recon_error.append(model.train_recon_error[-1])

        filename = 'data/{}_{}_{}_{}_{}.acc_f1.pkl'.format(model_name, model_lam, model_dim, model_epc, save_id)
        filehandler = open(filename, 'rb')
        evaluation = pickle.load(filehandler)
        filehandler.close()
        rf_acc.append(evaluation['RF-ACC'])
        rf_f1.append(evaluation['RF-F1'])
        svc_acc.append(evaluation['SVC-ACC'])
        svc_f1.append(evaluation['SVC-F1'])
        knn_acc.append(evaluation['KNN-ACC'])
        knn_f1.append(evaluation['KNN-F1'])
    data['RF-ACC'].append("{:.4f}±{:.4f}".format(np.mean(rf_acc), np.std(rf_acc)))
    data['RF-F1'].append("{:.4f}±{:.4f}".format(np.mean(rf_f1), np.std(rf_f1)))
    data['SVC-ACC'].append("{:.4f}±{:.4f}".format(np.mean(svc_acc), np.std(svc_acc)))
    data['SVC-F1'].append("{:.4f}±{:.4f}".format(np.mean(svc_f1), np.std(svc_f1)))
    data['KNN-ACC'].append("{:.4f}±{:.4f}".format(np.mean(knn_acc), np.std(knn_acc)))
    data['KNN-F1'].append("{:.4f}±{:.4f}".format(np.mean(knn_f1), np.std(knn_f1)))
print(tabulate(data, headers='keys', tablefmt='fancy_grid'))

╒══════════════════╤══════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Modelo           │   Lambda │ RF-ACC        │ RF-F1         │ SVC-ACC       │ SVC-F1        │ KNN-ACC       │ KNN-F1        │
╞══════════════════╪══════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ Conv_custom_dim3 │     0    │ 0.7799±0.0058 │ 0.7791±0.0062 │ 0.6756±0.0151 │ 0.7301±0.0153 │ 0.7593±0.0198 │ 0.7581±0.0196 │
├──────────────────┼──────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Conv_custom_dim3 │     0.1  │ 0.7842±0.0182 │ 0.7831±0.0184 │ 0.6984±0.0153 │ 0.7425±0.0134 │ 0.7504±0.0080 │ 0.7478±0.0086 │
├──────────────────┼──────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Conv_custom_dim3 │     0.01 │ 0.7730±0.0498 │ 0.7798±0.0340 │ 0.6951±0.0174 │ 0.7489±0.0180 │ 0.7447±0